In [23]:
%%capture
# NER для извлечения артикулов из описаний товаров

# 1. Установка и импорт библиотек
# Устанавливаем spaCy и необходимые зависимости. Используем модель для русского языка, так как запрос на русском.

# !pip install spacy
!python -m spacy download ru_core_news_sm

import random
import pandas as pd
from sklearn.model_selection import train_test_split


import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from spacy.scorer import Scorer

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_excel('выгрузка_артикулов.xlsx')
data.head()

,text,entity
0,"Тест-система для идентификации линий ГМО ""Соя ...",GM-202-50
1,Адаптер/муфта F603462/0,F603462/0
2,БОЛТ KG00468961,KG00468961
3,V ремень ksn1000 Holdijk Haamberg (106384),106384
4,Поршень гидроцилиндра № 0387304,0387304


In [4]:
# 2. Подготовка данных
# Преобразуем данные в формат, подходящий для spaCy

def prepare_training_data(data):
    training_data = []
    for index, row in data.iterrows():
        text = row['text']
        article = row['entity']
        # Находим позицию артикула в тексте
        start_idx = text.find(article)
        if start_idx != -1:
            end_idx = start_idx + len(article)
            entities = [(start_idx, end_idx, "ARTICLE")]
            training_data.append((text, {"entities": entities}))
    return training_data

training_data = prepare_training_data(data)
training_data[:5]

[('Тест-система для идентификации линий ГМО "Соя A2704-12 идентификация" (50 тестов) Артикул:Sintol-GM-202-50',
  {'entities': [(97, 106, 'ARTICLE')]}),
 ('Адаптер/муфта F603462/0', {'entities': [(14, 23, 'ARTICLE')]}),
 ('БОЛТ KG00468961', {'entities': [(5, 15, 'ARTICLE')]}),
 ('V ремень ksn1000 Holdijk Haamberg (106384)',
  {'entities': [(35, 41, 'ARTICLE')]}),
 ('Поршень гидроцилиндра № 0387304', {'entities': [(24, 31, 'ARTICLE')]})]

In [7]:
nlp = spacy.load("ru_core_news_sm")

In [13]:
# Разделяем на обучающую, валидационную и тестовую выборки
train_data, temp_data = train_test_split(training_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Напишем функцию оценки модели для ранней остновки при обучении
def evaluate_ner(nlp, data):
    scorer = Scorer()
    examples = []
    for text, annotations in data:
        doc = nlp(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)
    scores = scorer.score(examples)
    return scores['ents_f']

# 3. Настройка и обучение модели
# Загружаем базовую модель spaCy и добавляем новый тип сущности "ARTICLE".

# Загружаем модель для русского языка
nlp = spacy.load("ru_core_news_sm")
ner = nlp.get_pipe("ner")

# Добавляем метку "ARTICLE"
ner.add_label("ARTICLE")
print(ner.labels)

# Параметры early stopping
patience = 5  # Количество эпох без улучшения
min_delta = 0.01  # Минимальное улучшение F1-score
best_f1 = 0.0
epochs_without_improvement = 0
max_epochs = 50
best_model_path = "best_ner_model"

('ARTICLE', 'LOC', 'ORG', 'PER')


In [14]:
# Отключаем другие компоненты пайплайна
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for epoch in range(max_epochs):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001)) # minibatch разбивает данные на батчи
        # compounding создает последовательность размеров батчей, которая начинается с 4 (4.0), увеличивается до максимума 32 (32.0) с коэффициентом роста 1.001
        # Такой подход помогает модели сначала обучаться на маленьких батчах (для стабильности) и постепенно переходить к большим (для скорости)
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in zip(texts, annotations)]
            nlp.update(examples, drop=0.5, losses=losses)

        # Оценка на валидационной выборке
        f1_score = evaluate_ner(nlp, val_data)
        print(f"Эпоха {epoch + 1}, Потери: {losses['ner']:.3f}, F1-score (val): {f1_score:.3f}")

        # Early stopping
        if f1_score > best_f1 + min_delta:
            best_f1 = f1_score
            epochs_without_improvement = 0
            # Сохраняем лучшую модель
            nlp.to_disk(best_model_path)
            print(f"Сохранена лучшая модель с F1-score: {best_f1:.3f}")
        else:
            epochs_without_improvement += 1
            print(f"Эпох без улучшения: {epochs_without_improvement}/{patience}")

        if epochs_without_improvement >= patience:
            print(f"Early stopping на эпохе {epoch + 1}. Лучший F1-score: {best_f1:.3f}")
            break

Эпоха 1, Потери: 4567.196, F1-score (val): 0.895
Сохранена лучшая модель с F1-score: 0.895
Эпоха 2, Потери: 2071.501, F1-score (val): 0.917
Сохранена лучшая модель с F1-score: 0.917
Эпоха 3, Потери: 1602.124, F1-score (val): 0.923
Эпох без улучшения: 1/5
Эпоха 4, Потери: 1318.972, F1-score (val): 0.919
Эпох без улучшения: 2/5
Эпоха 5, Потери: 1096.162, F1-score (val): 0.930
Сохранена лучшая модель с F1-score: 0.930
Эпоха 6, Потери: 888.631, F1-score (val): 0.920
Эпох без улучшения: 1/5
Эпоха 7, Потери: 801.508, F1-score (val): 0.931
Эпох без улучшения: 2/5
Эпоха 8, Потери: 653.207, F1-score (val): 0.924
Эпох без улучшения: 3/5
Эпоха 9, Потери: 586.906, F1-score (val): 0.926
Эпох без улучшения: 4/5
Эпоха 10, Потери: 562.616, F1-score (val): 0.930
Эпох без улучшения: 5/5
Early stopping на эпохе 10. Лучший F1-score: 0.930


In [15]:
# 4. Загрузка лучшей модели
nlp = spacy.load(best_model_path)

# 5. Оценка на тестовой выборке
test_f1 = evaluate_ner(nlp, test_data)
print(f"\nФинальный F1-score на тестовой выборке: {test_f1:.3f}")


Финальный F1-score на тестовой выборке: 0.922


In [20]:
# 6. Тестирование на примерах
# Проверяем, как модель извлекает артикулы из новых текстов.
# Для примера возьмем пару наименований товаров из DNS

test_texts = [
    'Процессор AMD Ryzen 5 7500F',
    '16" Ноутбук Maibenben X16A-R77446 серый',
    'Кондиционер мобильный Timberk T-PAC09-P12E-WF белый, черный',
    'Корпус Cougar MX330-F черный'
]

for text in test_texts:
    doc = nlp(text)
    print(f"\nТекст: {text}")
    print("Извлеченные артикулы:")
    for ent in doc.ents:
        if ent.label_ == "ARTICLE":
            print(f"- {ent.text}")


Текст: Процессор AMD Ryzen 5 7500F
Извлеченные артикулы:
- 7500F

Текст: 16" Ноутбук Maibenben X16A-R77446 серый
Извлеченные артикулы:
- X16A-R77446

Текст: Кондиционер мобильный Timberk T-PAC09-P12E-WF белый, черный
Извлеченные артикулы:

Текст: Корпус Cougar MX330-F черный
Извлеченные артикулы:
- MX330-F
